# Fitting Logistic Regression

In [1]:
import os
os.chdir("/home/jacob/Project/LendingClub")

In [2]:
import Data_Helper as DH
import lendingclub
import prediction

import numpy as np
import pandas as pd
import json
import time
import sklearn
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

pd.options.mode.chained_assignment = None  # default='warn'
%load_ext autoreload
%autoreload 2

## Instantiate Objects

In [9]:
# initialize config object
config = lendingclub.ConfigData("config_data_dummy.ini")

# initialize lendingclup api object
lc = lendingclub.LendingClub(config)

# initialize data transformer
transformer = DH.Transformer_full()

# initialize DataHelper
periodStart = ("Q1", "2014")
periodEnd = ("Q3", "2016")
DataHelper = DH.DataHelper(periodStart, periodEnd, transformer, lc)

In [10]:
# set training data
DataHelper.set_training_dataset()

In [12]:
training.columns.values

array(['acc_now_delinq', 'acc_open_past_24mths', 'addr_state', 'all_util',
       'annual_inc', 'application_type', 'avg_cur_bal', 'bc_open_to_buy',
       'bc_util', 'chargeoff_within_12_mths',
       'collections_12_mths_ex_med', 'delinq_2yrs', 'delinq_amnt', 'dti',
       'earliest_cr_line', 'emp_length', 'home_ownership', 'il_util',
       'inq_fi', 'inq_last_12m', 'inq_last_6mths', 'installment',
       'int_rate', 'loan_amnt', 'max_bal_bc', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mort_acc', 'mths_since_rcnt_il', 'mths_since_recent_bc',
       'mths_since_recent_inq', 'num_accts_ever_120_pd', 'num_actv_bc_tl',
       'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl',
       'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0',
       'num_sats', 'num_tl_120dpd_2m', 'num_tl_30dpd',
       'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'open_acc',
       'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m

In [11]:
training = DataHelper.training
training.shape

(666934, 94)

In [5]:
# set test data
DataHelper.set_test_dataset(["LoanStats_2016Q3.csv", "LoanStats_2016Q4.csv"])

## Logistic Regression - choose penalization type / solver

Based on the test run, l1/saga, l2/saga, and l2sag seem to be appropriat choices. We will try to finetune C parameter for these three pairs.

In [5]:
# logstic regression model
logistic_model = prediction.ModelLogistic()

In [6]:
# get training/test for this model
training, test = logistic_model.get_data_for_model(DataHelper.training, DataHelper.test)

In [7]:
# Check time - l1/saga
start = time.time()

logistic_model.solver = "saga"
logistic_model.penalty = "l1"

kwargs = {"C":1.0}
model = logistic_model.fit_model(training, **kwargs)
score = logistic_model.test_model(test, model)
print("Raw test score:",score)

end = time.time()
print(end - start)

Raw test score: {'score': 0.7678555719549982, 'AUC': 0.67946877859616}
294.1440062522888


In [17]:
# Check time - l2/sag
start = time.time()

logistic_model.solver = "sag"
logistic_model.penalty = "l2"

kwargs = {"C":1.0}
model = logistic_model.fit_model(training, **kwargs)
score = logistic_model.test_model(test, model)
print("Raw test score:",score)

end = time.time()
print(end - start)

Raw test score: {'score': 0.7656695802962022, 'AUC': 0.6801530740858839}
240.99648475646973


In [18]:
# Check time - l2/lbfgs
start = time.time()

logistic_model.solver = "lbfgs"
logistic_model.penalty = "l2"
logistic_model.max_iter = 500

kwargs = {"C":1.0}
model = logistic_model.fit_model(training, **kwargs)
score = logistic_model.test_model(test, model)
print("Raw test score:",score)

end = time.time()
print(end - start)

Raw test score: {'score': 0.7656594985331034, 'AUC': 0.6801518382947149}
40.51711344718933


/home/jacob/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [20]:
# Check time - l2/lbfgs
start = time.time()

logistic_model.solver = "saga"
logistic_model.penalty = "l2"
logistic_model.max_iter = 500

kwargs = {"C":1.0}
model = logistic_model.fit_model(training, **kwargs)
score = logistic_model.test_model(test, model)
print("Raw test score:",score)

end = time.time()
print(end - start)

Raw test score: {'score': 0.7656695802962022, 'AUC': 0.6801559992058404}
238.20778250694275


## Logistic Regression - CV

In [9]:
# get CV index from DataHelper
CVs = DataHelper.get_cross_validation_data(fold=5)

We first try l1 penalization with saga solver. Max_iteration is set to be 500.

In [10]:
logistic_model = prediction.ModelLogistic()
logistic_model.solver = "saga"
logistic_model.penalty = "l1"
logistic_model.max_iter = 500

In [11]:
# choose grids to fit
grids = [{"C": x} for x in [10000, 1000, 100, 10, 1, 0.1, 0.01, 0.001]]

In [12]:
scores = logistic_model.tune_parameters(DataHelper.training, CVs, grids, verbose=True)

fitting for {'C': 10000}


/home/jacob/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jacob/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/jacob/Project/LendingClub/prediction.py:168: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  test.loc[:, cvars] = scaler.transform(test[cvars])


{'score': 0.8445823937157265, 'AUC': 0.7008842315175903}


/home/jacob/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jacob/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/jacob/Project/LendingClub/prediction.py:168: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  test.loc[:, cvars] = scaler.transform(test[cvars])
/home/jacob/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


{'score': 0.8457634322574193, 'AUC': 0.700992980385214}


...

{'score': 0.8524278958236196, 'AUC': 0.6951281300357296}


/home/jacob/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jacob/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/jacob/Project/LendingClub/prediction.py:168: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  test.loc[:, cvars] = scaler.transform(test[cvars])


{'score': 0.8512138670574131, 'AUC': 0.6916722237583557}


/home/jacob/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jacob/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/jacob/Project/LendingClub/prediction.py:168: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  test.loc[:, cvars] = scaler.transform(test[cvars])


{'score': 0.851192408540392, 'AUC': 0.6957552307326609}


In [13]:
# find best param
_, avg_score, param = logistic_model.best_grid(scores, "AUC")
print(avg_score)
print(param)

0.6995973698198783
{'C': 10000}


For the next, we try l2 penalization with saga solver. Max_iteration is set to be 500.

In [14]:
logistic_model = prediction.ModelLogistic()
logistic_model.solver = "saga"
logistic_model.penalty = "l2"
logistic_model.max_iter = 500

In [15]:
# choose grids to fit
grids = [{"C": x} for x in [10000, 1000, 100, 10, 1, 0.1, 0.01, 0.001]]

In [16]:
scores = logistic_model.tune_parameters(DataHelper.training, CVs, grids, verbose=True)

fitting for {'C': 10000}


/home/jacob/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jacob/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/jacob/Project/LendingClub/prediction.py:168: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  test.loc[:, cvars] = scaler.transform(test[cvars])


{'score': 0.8445990363892055, 'AUC': 0.7008823557005057}


/home/jacob/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jacob/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/jacob/Project/LendingClub/prediction.py:168: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  test.loc[:, cvars] = scaler.transform(test[cvars])


{'score': 0.8457801001741797, 'AUC': 0.7010220817113708}


...

{'score': 0.8524445480583495, 'AUC': 0.6986281763714413}


/home/jacob/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jacob/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/jacob/Project/LendingClub/prediction.py:168: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  test.loc[:, cvars] = scaler.transform(test[cvars])


{'score': 0.8512471693086453, 'AUC': 0.6952887065823692}


/home/jacob/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jacob/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/jacob/Project/LendingClub/prediction.py:168: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  test.loc[:, cvars] = scaler.transform(test[cvars])


{'score': 0.8513755358554959, 'AUC': 0.6984971055932568}


In [17]:
# find best param
_, avg_score, param = logistic_model.best_grid(scores, "AUC")
print(avg_score)
print(param)

0.6995964904345755
{'C': 10000}


We first try l2 penalization with sag solver. Max_iteration is set to be 500.

In [18]:
logistic_model = prediction.ModelLogistic()
logistic_model.solver = "sag"
logistic_model.penalty = "l2"
logistic_model.max_iter = 500

In [19]:
# choose grids to fit
grids = [{"C": x} for x in [10000, 1000, 100, 10, 1, 0.1, 0.01, 0.001]]

In [20]:
scores = logistic_model.tune_parameters(DataHelper.training, CVs, grids, verbose=True)

fitting for {'C': 10000}


/home/jacob/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jacob/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/jacob/Project/LendingClub/prediction.py:168: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  test.loc[:, cvars] = scaler.transform(test[cvars])


{'score': 0.8445823937157265, 'AUC': 0.7008841777873622}


/home/jacob/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jacob/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/jacob/Project/LendingClub/prediction.py:168: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  test.loc[:, cvars] = scaler.transform(test[cvars])
/home/jacob/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:334: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


{'score': 0.8457634322574193, 'AUC': 0.7009929942315766}


...

{'score': 0.8524445480583495, 'AUC': 0.6986277795860903}


/home/jacob/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jacob/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/jacob/Project/LendingClub/prediction.py:168: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  test.loc[:, cvars] = scaler.transform(test[cvars])


{'score': 0.8512471693086453, 'AUC': 0.6952888555513774}


/home/jacob/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/jacob/anaconda3/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/jacob/Project/LendingClub/prediction.py:168: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  test.loc[:, cvars] = scaler.transform(test[cvars])


{'score': 0.8513755358554959, 'AUC': 0.6984965909129444}


In [21]:
# find best param
_, avg_score, param = logistic_model.best_grid(scores, "AUC")
print(avg_score)
print(param)

0.6995974494401943
{'C': 10000}


## Fit Best Model & Save

In [5]:
# fit best model
logistic_model = prediction.ModelLogistic()
logistic_model.solver = "saga"
logistic_model.penalty = "l1"
logistic_model.max_iter = 500

In [6]:
training, test = logistic_model.get_data_for_model(DataHelper.training, DataHelper.test)

In [7]:
kwargs = {"C":10000}
model = logistic_model.fit_model(training, **kwargs)
score = logistic_model.test_model(test, model)

In [8]:
score

{'score': 0.7678457974527647, 'AUC': 0.6794262476378288}

In [9]:
scaler = logistic_model.get_scaler(DataHelper.training)

In [10]:
# save model

In [11]:
filename = "logistic_regression_20190610.sav"

In [12]:
logistic_model.save_model(model,scaler,training,filename)

Model is saved on logistic_regression_20190610.sav


## Predict listed loan by saved model

In [5]:
# predict by calling model

In [6]:
filename = "logistic_regression_20190610.sav"

In [7]:
# instantiate from beginning
new_model = prediction.ModelLogistic(filename)

In [8]:
# update from the saved model
new_model.set_model_from_file()

Model is loaded from logistic_regression_20190610.sav


In [9]:
# check model description
new_model.model_description

'Logistic Regression Object, time: 08/19/2019, C: 10000, penalty: l1, solver: saga, max_iter: 500, tol: 0.0005'

In [10]:
# get listed loan
loans = DataHelper.get_listed_loandata()
loans.shape

HTTPError: 401 Client Error: Unauthorized for url: https://api.lendingclub.com/api/investor/v1/loans/listing?showAll=true

In [37]:
# prediction
pred = new_model.predict_model(loans)
pred.shape

/home/jacob/Project/LendingClub/prediction.py:313: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  df.loc[:, cvars] = self.scaler.transform(df[cvars])


(65, 2)